In [1]:
import pandas as pd
import geopandas as gpd

from pathlib import Path
data = Path('data/data_for_model')

In [2]:
%ls data | grep feather

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
attrs = gpd.read_feather(f'data/temp/far_bsc_primary.feather')

In [5]:
grid = gpd.read_feather(f'{data}/grid.feather')

In [6]:
grid.shape

(92596, 6)

In [7]:
attrs.shape

(435479, 13)

In [8]:
grid.head()

,id_x,title,idx,ward_num,name,geometry
0,"(47965, 87513, 18)","XYZ tile (47965, 87513, 18)",4.796588e+11,2,02L,"POLYGON ((281388.362 5678062.191, 281392.439 5..."
1,"(47966, 87513, 18)","XYZ tile (47966, 87513, 18)",4.796688e+11,2,02L,"POLYGON ((281484.255 5678058.105, 281488.329 5..."
2,"(47967, 87513, 18)","XYZ tile (47967, 87513, 18)",4.796788e+11,2,02L,"POLYGON ((281580.147 5678054.020, 281584.220 5..."
3,"(47968, 87513, 18)","XYZ tile (47968, 87513, 18)",4.796888e+11,2,02L,"POLYGON ((281676.040 5678049.937, 281680.111 5..."
4,"(47969, 87513, 18)","XYZ tile (47969, 87513, 18)",4.796988e+11,2,02L,"POLYGON ((281771.933 5678045.856, 281776.002 5..."


In [9]:
attrs = attrs.to_crs(epsg=32612)

In [10]:
attrs.area

3         1819.194151
4          252.852837
9          280.625098
10         280.464699
11         958.281089
             ...     
374576     305.671823
374613     256.257866
374617     863.764459
374618     780.298167
374621     280.060840
Length: 435479, dtype: float64

In [11]:
attrs.head()

,SHAPE_Area,geometry,index_right,STRUCTURE_ID,BUILDING_HEIGHT_MIN,SUBCLASS,SPACE_AYOC,SHAPE_Area_y,floor,floor_number,floor_area,far,bsc
3,1818.213538,"MULTIPOLYGON (((284587.031 5658753.918, 284541...",57280.0,1397233.0,9.800000,Residential,1962.0,432.311493,1.225000,1.0,432.311493,0.237767,0.237767
4,252.704619,"MULTIPOLYGON (((279978.049 5660948.528, 280011...",959007.0,2511967.0,6.792218,Residential,2018.0,144.041183,0.849027,1.0,144.041183,0.569998,0.569998
9,280.457889,"MULTIPOLYGON (((279062.583 5657040.859, 279064...",994809.0,2364500.0,6.792218,Residential,2015.0,172.310664,0.849027,1.0,172.310664,0.614391,0.614391
10,280.297608,"MULTIPOLYGON (((279072.035 5657075.524, 279080...",958276.0,2466405.0,3.478505,Others,2015.0,74.420055,0.496929,1.0,74.420055,0.265504,0.265504
11,957.741683,"MULTIPOLYGON (((282271.324 5658588.122, 282271...",1672482.0,1005078.0,8.750000,Commercial,1952.0,235.839073,1.250000,1.0,235.839073,0.246245,0.246245


In [12]:
attrs = (attrs.assign(centroid=lambda x: x.centroid)
              .rename(columns={'geometry': 'parcel_boundary'})
              .set_geometry('centroid'))

In [13]:
attrs.head()

,SHAPE_Area,parcel_boundary,index_right,STRUCTURE_ID,BUILDING_HEIGHT_MIN,SUBCLASS,SPACE_AYOC,SHAPE_Area_y,floor,floor_number,floor_area,far,bsc,centroid
3,1818.213538,"MULTIPOLYGON (((284587.031 5658753.918, 284541...",57280.0,1397233.0,9.800000,Residential,1962.0,432.311493,1.225000,1.0,432.311493,0.237767,0.237767,POINT (284565.969 5658775.710)
4,252.704619,"MULTIPOLYGON (((279978.049 5660948.528, 280011...",959007.0,2511967.0,6.792218,Residential,2018.0,144.041183,0.849027,1.0,144.041183,0.569998,0.569998,POINT (279994.464 5660944.016)
9,280.457889,"MULTIPOLYGON (((279062.583 5657040.859, 279064...",994809.0,2364500.0,6.792218,Residential,2015.0,172.310664,0.849027,1.0,172.310664,0.614391,0.614391,POINT (279067.309 5657058.191)
10,280.297608,"MULTIPOLYGON (((279072.035 5657075.524, 279080...",958276.0,2466405.0,3.478505,Others,2015.0,74.420055,0.496929,1.0,74.420055,0.265504,0.265504,POINT (279075.300 5657057.819)
11,957.741683,"MULTIPOLYGON (((282271.324 5658588.122, 282271...",1672482.0,1005078.0,8.750000,Commercial,1952.0,235.839073,1.250000,1.0,235.839073,0.246245,0.246245,POINT (282255.932 5658596.401)


In [14]:
attrs['year'] = pd.cut(attrs.SPACE_AYOC, bins=[0, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020], labels=[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [15]:
attrs_by_year = {year: grp for year, grp in attrs.groupby('year')}

In [16]:
attrs_by_year[2013].head()

,SHAPE_Area,parcel_boundary,index_right,STRUCTURE_ID,BUILDING_HEIGHT_MIN,SUBCLASS,SPACE_AYOC,SHAPE_Area_y,floor,floor_number,floor_area,far,bsc,centroid,year
3,1818.213538,"MULTIPOLYGON (((284587.031 5658753.918, 284541...",57280.0,1397233.0,9.800000,Residential,1962.0,432.311493,1.225000,1.0,432.311493,0.237767,0.237767,POINT (284565.969 5658775.710),2013
11,957.741683,"MULTIPOLYGON (((282271.324 5658588.122, 282271...",1672482.0,1005078.0,8.750000,Commercial,1952.0,235.839073,1.250000,1.0,235.839073,0.246245,0.246245,POINT (282255.932 5658596.401),2013
12,1155.865770,"MULTIPOLYGON (((279563.975 5674007.345, 279557...",1666995.0,1587676.0,6.792218,Residential,2013.0,492.244818,0.849027,1.0,492.244818,0.425867,0.425867,POINT (279561.651 5674020.450),2013
24,278.555030,"MULTIPOLYGON (((285859.778 5662576.655, 285858...",998218.0,2330072.0,6.792218,Residential,2013.0,169.803595,0.849027,1.0,169.803595,0.609587,0.609587,POINT (285855.233 5662558.540),2013
35,326.947327,"MULTIPOLYGON (((283977.676 5665520.851, 283946...",823739.0,1141032.0,4.140000,Residential,1967.0,217.050462,0.517500,1.0,217.050462,0.663870,0.663870,POINT (283964.245 5665532.007),2013


In [17]:
attrs = attrs.drop(columns='index_right')

In [18]:
grid_attrs = {}

def mode(s):
    if s.value_counts().empty: return 0
    else: return s.value_counts().index[0]

def process(year):
    df = gpd.sjoin(left_df=grid, right_df=attrs[attrs.year <= year], how='left', op='intersects')
    df = df.groupby('idx').agg({
        'SHAPE_Area': ['mean', 'median', 'sum'],
        'SHAPE_Area_y': ['mean', 'median', 'sum','count'],
        'floor_area': ['mean', 'median', 'sum'],
        'far': ['mean', 'median', 'sum'],
        'bsc': ['mean', 'median', 'sum'],
        'BUILDING_HEIGHT_MIN': ['mean', 'median', 'sum'],
        'parcel_boundary': 'count',
        'floor_number': ['sum', 'mean', 'median'],
        'floor_number': ['sum', 'mean', 'median', mode],
        'SUBCLASS': [mode]
    }).reset_index()

    df.columns = ['_'.join(c) for c in df.columns.values]
    df['year'] = year
    
    return df
    

for year in range(2013, 2021):
    print(f'Processing... {year}')
    grid_attrs[year] = process(year)

Processing... 2013
Processing... 2014
Processing... 2015
Processing... 2016
Processing... 2017
Processing... 2018
Processing... 2019
Processing... 2020


In [19]:
grid_attrs_yearly = pd.concat([df for year, df in grid_attrs.items()])

In [20]:
grid_attrs_yearly.rename(columns={'idx_': 'idx'}, inplace=True)

In [21]:
grid_attrs_yearly.sort_values(by=['idx', 'year'], inplace=True)

In [22]:
grid_attrs_yearly = grid_attrs_yearly.reset_index(drop=True)

In [23]:
grid_attrs_yearly.SUBCLASS_mode = grid_attrs_yearly.SUBCLASS_mode.astype('str')

In [24]:
grid_attrs_yearly.to_feather(f'{data}/attrs_yearly.feather')